In [0]:
# Mount Google Driver
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive
# change to clrs directionary
%cd "/content/drive/My Drive/Colab Notebooks/fluent_python_notes"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/fluent_python_notes


In [0]:
%mkdir ch13
!touch ch13/__init__.py

## 13.0 序论

- 运算符重载的的作用是让用户定义的对象中支持中缀运算符（如 `＋` 和 `｜`） 或一元运算符(如 `-` 和 `~`)

## 13.1 运算符重载基础

- Python 对运算符重载施加了一些限制,做好了灵活性、可用性和安全性方面的平衡
  - 不能重载内置类型的运算符
  - 不能新建运算符,只能重载现有的
  - 某些运算符不能重载—— `is`、`and`、`or` 和 `not`(不过位运算符 `&`、`|` 和 `~` 可以)

## 13.2 一元运算符

- 一元运算符及其对应的特殊方法
  - `-` (`__neg__`)
    - 一元取负运算符
  - `+` (`__pos__`)
    - 一元取正运算符
  - `~` (`__invert__`)
    - 对整数按位取反
  - `abs()` (`__abs__`)
    - 取绝对值
- 一元运算符的基本原则
  - 始终返回一个新对象
  - 即不能修改 self, 要创建并返回合适类型的新类型

###### 示例 13-1 vector_v6.py:把一元运算符 `-` 和 `+` 添加到示例 10-16 中

In [0]:
!cat ch10/vector_v5.py > ch13/vector_v6.py

In [15]:
%%writefile -a ch13/vector_v6.py
  
  
  def __neg__(self):
    return Vector(-x for x in self)

  def __pos__(self):
    return Vector(self)  # 构建一个新的实例


Appending to ch13/vector_v6.py


### `x` 与 `++x` 何时不相等

###### 示例 13-2 算术运算符上下文的变化可能导致 `x` 不等于 `+x`

In [17]:
import decimal
ctx = decimal.getcontext()  # 获取当前全局算术运算的上下文引用
ctx.prec = 40  # 把算术运算上下文的精度设为 40
one_third = decimal.Decimal('1') / decimal.Decimal('3')
one_third

Decimal('0.3333333333333333333333333333333333333333')

In [18]:
one_third == +one_third

True

In [19]:
ctx.prec = 28  # 把精度降低为 28,这是 Python 3.4 为 Decimal 算术运算设定的默认精度
one_third == +one_third

False

In [20]:
+one_third

Decimal('0.3333333333333333333333333333')

###### 示例 13-3 一元运算符 `+` 得到一个新 `Counter` 实例,但是没有零值和负值计数器

- [collections.Counter](https://docs.python.org/3/library/collections.html#collections.Counter) 中的中缀运算符的作用是把两个 `Counter` 实例的计数器加在一起
- `Counter` 相加时,负值和零值计数会从结果中剔除
- 一元运算符 `+` 等同于加上一个空 `Counter`, 因此它会产生一个新的 `Counter` 且仅保留大于零的计数器

In [22]:
from collections import Counter
ct = Counter('abracadabra')
ct

Counter({'a': 5, 'b': 2, 'c': 1, 'd': 1, 'r': 2})

In [23]:
ct['r'], ct['d'] = -3, 0
ct

Counter({'a': 5, 'b': 2, 'c': 1, 'd': 0, 'r': -3})

In [24]:
+ct

Counter({'a': 5, 'b': 2, 'c': 1})

## 13.3 重载向量加法运算符

###### 示例 13-4 `Vector.__add__` 方法,第 1 版

In [0]:
from ch13.vector_v6 import Vector

In [0]:
import itertools

def add_v1(self, other):
  pairs = itertools.zip_longest(self, other, fillvalue=0.0)
  return Vector(a + b for a, b in pairs)

Vector.__add__ = add_v1

###### 示例 13-5 第 1 版 `Vector.__add__` 方法也支持 `Vector` 之外的对象

In [29]:
v1 = Vector([3, 4, 5])
v1 + (10, 20, 30)

Vector([13.0, 24.0, 35.0])

In [31]:
from ch9.vector2d_v3 import Vector2d
v2d = Vector2d(1, 2)
v1 + v2d

Vector([4.0, 6.0, 5.0])

###### 示例 13-6 如果左操作数是 `Vector` 之外的对象,第一版 `Vector.__add__` 方法无法处理